# Simple Convnet - MNIST

Slightly modified from mnist_cnn.py in the Keras examples folder:

https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py

In [1]:
KERAS_MODEL_FILEPATH = '../../demos/data/mnist_cnn/mnist_cnn.h5'

In [2]:
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.callbacks import EarlyStopping, ModelCheckpoint

Using TensorFlow backend.
/home/leon/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [3]:
num_classes = 10

# input image dimensions
img_rows, img_cols = 28, 28

# the data, shuffled and split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = np_utils.to_categorical(y_train, num_classes)
y_test = np_utils.to_categorical(y_test, num_classes)

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [4]:
# Sequential Model

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [5]:
# Model saving callback
checkpointer = ModelCheckpoint(filepath=KERAS_MODEL_FILEPATH, monitor='val_acc', verbose=1, save_best_only=True)

# Early stopping
early_stopping = EarlyStopping(monitor='val_acc', verbose=1, patience=5)

# Train
batch_size = 128
epochs = 100
model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=2,
          callbacks=[checkpointer, early_stopping], 
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test score:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/100
Epoch 00001: val_acc improved from -inf to 0.98270, saving model to mnist_cnn.h5
 - 4s - loss: 0.2833 - acc: 0.9127 - val_loss: 0.0557 - val_acc: 0.9827
Epoch 2/100
Epoch 00002: val_acc improved from 0.98270 to 0.98700, saving model to mnist_cnn.h5
 - 2s - loss: 0.0980 - acc: 0.9708 - val_loss: 0.0407 - val_acc: 0.9870
Epoch 3/100
Epoch 00003: val_acc improved from 0.98700 to 0.98870, saving model to mnist_cnn.h5
 - 2s - loss: 0.0747 - acc: 0.9777 - val_loss: 0.0344 - val_acc: 0.9887
Epoch 4/100
Epoch 00004: val_acc improved from 0.98870 to 0.98960, saving model to mnist_cnn.h5
 - 2s - loss: 0.0611 - acc: 0.9811 - val_loss: 0.0309 - val_acc: 0.9896
Epoch 5/100
Epoch 00005: val_acc did not improve
 - 2s - loss: 0.0542 - acc: 0.9836 - val_loss: 0.0329 - val_acc: 0.9888
Epoch 6/100
Epoch 00006: val_acc improved from 0.98960 to 0.99140, saving model to mnist_cnn.h5
 - 2s - loss: 0.0483 - acc: 0.9853 - val_loss: 0.0289 - val_acc: